In [3]:

import jax
import jax.numpy as jnp
import flax
import tokenizer

# from tokenizer import tokenizeLine
# tokenizer.tokenize()

vocab = tokenizer.makeVocabUCI()
games = open('data/ELO_2000_UCI.txt', 'r').read()
games = games.splitlines()

tokenizedGames = []

for g in games:
    arr = []
    for e in g.split(' '):
        arr.append(vocab[e])
    tokenizedGames.append(arr)

In [41]:
import jax
import jax.numpy as jnp
import flax.linen as nn
randKEY = jax.random.PRNGKey(seed=123)

test = jnp.array([1,3,3,3,2,0,4,5,0,7,8,9,10,11,12,13,14])

def splitGame(x:jnp.array, padToken):
    global randKEY
    ind = jnp.where(x == padToken)[0][0]
    
    randKEY, k = jax.random.split(randKEY)
    idx = jax.random.randint(k, (1,), 2, ind)[0]
    print(ind, 'with split at', idx)
    maskY = jnp.where(jnp.arange(x.shape[0]) <= idx, 1, 0)
    # print(maskY)
    maskX = jnp.where(jnp.arange(x.shape[0]) < idx, 1, 0)
    # print(maskX)

    # print(ind)
    # print(x[ind])
    # print(x)
    # print(mask)
    return x*maskX, x*maskY
for i in range(3):
    d, t = splitGame(test, 0)
    print(type(d), type(t))
    print(d)
    print(t)
    print('---------------------')


5 with split at 3
<class 'jaxlib.xla_extension.ArrayImpl'> <class 'jaxlib.xla_extension.ArrayImpl'>
[1 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 3 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0]
---------------------
5 with split at 2
<class 'jaxlib.xla_extension.ArrayImpl'> <class 'jaxlib.xla_extension.ArrayImpl'>
[1 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
---------------------
5 with split at 3
<class 'jaxlib.xla_extension.ArrayImpl'> <class 'jaxlib.xla_extension.ArrayImpl'>
[1 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 3 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0]
---------------------
